# Charged Gel

In [1]:
from lammps import PyLammps

In [4]:
L = PyLammps()

LAMMPS (3 Mar 2020)
  using 16 OpenMP thread(s) per MPI task
LAMMPS output is captured by PyLammps wrapper


#### SET SIMULATION CONDITIONS

In [5]:
L.command("units         lj")
L.command("atom_style    full")
L.command("bond_style harmonic")
L.command("pair_style lj/expand 2.5")
L.command("kspace_style pppm 1.0e-4")

L.command("dimension     3")
L.command("boundary p p p")
L.command("neighbor 0.8 bin")

In [6]:
L.command("read_data neutralMicelles.data")
L.command("group charged type 2")

Reading data file ...
  orthogonal box = (-40 -40 -40) to (40 40 40)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  10827 atoms
  reading velocities ...
  10827 velocities
  scanning bonds ...
  2 = max bonds/atom
  reading bonds ...
  10800 bonds
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:   0          0          0         
  special bond factors coul: 0          0          0         
  4 = max # of 1-2 neighbors
  4 = max # of 1-3 neighbors
  8 = max # of 1-4 neighbors
  12 = max # of special neighbors
  special bonds CPU = 0.00281501 secs
  read_data CPU = 0.0340519 secs
8640 atoms in group charged


####  VARIABLES + SETUP

In [7]:
L.command("variable Temp equal 1.0")
L.command("variable dt equal 0.005")
L.command("variable Tdamping equal 100*${dt}")

L.command("thermo  20")
L.command("thermo_style    custom step temp ebond epair ke etotal enthalpy vol press")

L.command("velocity     all create ${Temp} 2398481")
L.command("timestep     ${dt}")

#### LANGEVIN

In [8]:
L.command("set type 2 charge -0.001")
L.command("variable rampCharge equal ramp(0.001,-1.0)")
L.command("fix charging charged adapt 10 atom charge v_rampCharge")

L.command("pair_style hybrid/overlay lj/cut 2.5 coul/long 10.0")
L.command("pair_coeff * * lj/cut 5.0 1.0 2.5")
L.command("pair_coeff 1 2 lj/cut 1.0 1.0 1.12")
L.command("pair_coeff 2 * lj/cut 1.0 1.0 1.12")
L.command("pair_coeff * * coul/long")

L.command("kspace_style pppm 1.0e-4")

L.command("fix nve1 all nve")
L.command("fix solvent all langevin 1.0 1.0 100.0 8434572")

L.command("dump dumpCharges all custom 20 Conf.dat id type xu yu zu vx vy vz")
L.command("dump_modify dumpCharges sort id")

L.command("reset_timestep 0")
L.command("run 2000")

Setting atom values ...
  8640 settings made for charge
PPPM initialization ...
  using 12-bit tables for long-range coulomb (src/kspace.cpp:332)
  G vector (1/distance) = 0.242389
  grid = 24 24 24
  stencil order = 5
  estimated absolute RMS force accuracy = 2.05018e-09
  estimated relative force accuracy = 2.05018e-09
  using double precision FFTW3
  3d grid and FFT values/proc = 24389 13824
Neighbor list info ...
  update every 1 steps, delay 10 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 10.8
  ghost atom cutoff = 10.8
  binsize = 5.4, bins = 15 15 15
  2 neighbor lists, perpetual/occasional/extra = 2 0 0
  (1) pair lj/cut, perpetual
      attributes: half, newton on
      pair build: half/bin/newton
      stencil: half/bin/3d/newton
      bin: standard
  (2) pair coul/long, perpetual, copy from (1)
      attributes: half, newton on
      pair build: copy
      stencil: none
      bin: none
Setting up Verlet run ...
  Unit style  